In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(10000)
x_test, y_test = parallel_dataset_generation(10000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9995829828000531, 0.9996489879174472)

In [7]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [8]:
pd.DataFrame(y)

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
0,NaN,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
1,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,NaN,ProvinceCode,Region,Municipality,...,Name,Surname,String,EMail,PhoneNumber,NaN,NaN,BiologicalSex,Boolean,NumericId
2,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NaN
3,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
4,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,NaN,EMail,PhoneNumber,Currency,NaN,BiologicalSex,Boolean,NaN
5,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,Name,Surname,String,EMail,NaN,Currency,Date,BiologicalSex,NaN,NumericId
6,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,NaN,EMail,PhoneNumber,NaN,Date,NaN,Boolean,NumericId
7,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,NaN,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,NaN,EMail,PhoneNumber,Currency,Date,BiologicalSex,NaN,NumericId
8,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,NaN,NaN,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
9,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,Name,NaN,String,EMail,PhoneNumber,Currency,NaN,NaN,Boolean,NaN


In [9]:
model.predict_dataframe(X)

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
0,NaN,ItalianVAT,NaN,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
1,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,NaN,ProvinceCode,Region,Municipality,...,Name,Surname,String,EMail,PhoneNumber,NaN,NaN,BiologicalSex,Boolean,NumericId
2,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NaN
3,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,NaN,String,EMail,PhoneNumber,Currency,Date,BiologicalSex,Boolean,NumericId
4,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,NaN,EMail,PhoneNumber,Currency,NaN,BiologicalSex,Boolean,NaN
5,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,Name,Surname,String,EMail,NaN,Currency,Date,BiologicalSex,NaN,NumericId
6,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,NaN,EMail,PhoneNumber,NaN,Date,NaN,Boolean,NumericId
7,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,NaN,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,Name,Surname,NaN,EMail,PhoneNumber,Currency,Date,BiologicalSex,NaN,NumericId
8,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,NaN,NaN,...,Name,Surname,String,EMail,PhoneNumber,NaN,Date,BiologicalSex,Boolean,NumericId
9,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,Name,NaN,String,EMail,PhoneNumber,Currency,NaN,NaN,Boolean,NaN


In [10]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('NaN', 'CountryCode'): 177,
         ('CountryCode', 'NaN'): 118,
         ('CountryCode', 'ProvinceCode'): 13,
         ('Surname', 'String'): 14,
         ('String', 'Name'): 42,
         ('String', 'Surname'): 8,
         ('Name', 'String'): 107,
         ('ItalianZIPCode', 'Currency'): 2,
         ('ProvinceCode', 'CountryCode'): 2,
         ('NumericId', 'Integer'): 35})